# ITR Schedule FA - Automated Data Entry

Automatically enter data from an CSV file to the ITR Schedule FA in the ITR-2 form using Selenium.

---

**Disclaimer:**

This script is provided for automation purposes only. Use it at your own risk. The developer is not responsible for any issues or damages that may arise from using this script.

## Pre-requisites

- **Python 3.9**: You can download it from [here](https://www.python.org/downloads/release/python-396/)
- **Selenium**: You can install it using the following command: `pip install selenium==4.10.0`
- **Pandas**: You can install it using the following command: `pip install pandas`

Additionally, you will need to have the webdriver for your browser. You can download the Chrome webdriver from [**chromium.org**](https://chromedriver.chromium.org/downloads) or [**googlechromelabs.github.io**](https://googlechromelabs.github.io/chrome-for-testing/#stable).

On macOS, you can install the Chrome webdriver using the following command: `brew install chromedriver`.

---

You can create a virtual environment and install the required packages using the following commands:


```shell
mkdir "schedule_fa_automated"

cd "schedule_fa_automated"

python -m venv .venv

source .venv/bin/activate

pip install selenium==4.10.0 pandas
```

To install jupyter notebook on virtual environment, you can use the following command:

```shell
pip install jupyter
```

To start the jupyter notebook, you can use the following command:

```shell
jupyter notebook
```

---

If you are using Jupyter Notebook, you can install the required packages using the following commands:

```shell
!pip install selenium==4.10.0 pandas
```

---

**Note**:
- Make sure that you have both browser and driver for the **same version**.


Consider the following variables that need to be set before running the script

**On Windows:**

| Variable | Description                                                     | Example                                           |
| --- |-----------------------------------------------------------------|---------------------------------------------------|
| `CHROME_DRIVER_PATH` | Local path to the downloaded webdriver **chromedriver.exe**     | `r"C:\Files\chromedriver_win32\chromedriver.exe"` |
| `BROWSER_PATH` | Local path to the **chrome.exe**                                | `r"C:\Files\Google\Chrome\Application\chrome.exe"` |
| `FILE_PATH` | Full path to the `CSV` file containing Schedule FA details      | `r"C:\Files\schedule_fa.csv"`                     |

**On macOS:**

| Variable | Description                                                 | Example                                         |
| --- |-------------------------------------------------------------|-------------------------------------------------|
| `CHROME_DRIVER_PATH` | Local path to the downloaded webdriver **chromedriver** file | `r"/usr/local/bin/chromedriver"`                   |
| `BROWSER_PATH` | Local path to the **Google Chrome**                     | `r"/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"` |
| `FILE_PATH` | Full path to the `CSV` file containing Schedule FA details  | `r"/Users/username/schedule_fa.csv"`              |


---

**Input CSV template:**

The CSV file should contain the following columns:

| Country Name | Country Code | Name of entity | Address of entity | zip code | Nature of entity | Date of acquiring interest/ stake | Initial value of investment | Peak value of investment | Closing Balance | Total gross amount paid/credited with respect to the holding during the period | Total gross proceeds from sale or redemption of investment during the period |
|--------------|--------------|----------------|-------------------|----------|------------------|----------------------------------|-----------------------------|-------------------------|-----------------|---------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|
| UNITED STATES| 2 - UNITED STATES | ABC Corp | 123 Main St | 12345 | Company | 2021-01-01 | 1000 | 2000 | 1500 | 500 | 100 |

## Constants

Based on your operating system (Windows or macOS), set the following variables:

In [ ]:
CHROME_DRIVER_PATH = r"YOUR_CHROME_DRIVER_PATH"
BROWSER_PATH = r"YOUR_CHROME_BROWSER_PATH"
FILE_PATH = r"YOUR_SCHEDULE_FA_REPORT_FILE_PATH"

# Country code for UNITED STATES in the ITR portal
COUNTRY = "2 - UNITED STATES"

verify the file exists at the specified path.

In [ ]:
!ls $FILE_PATH # for macOS
# !dir $FILE_PATH # for Windows

## Read CSV report

In [ ]:

# Based on the fields in the CSV file, you can define the fields here
class Fields:
    COUNTRY_NAME = "Country Name"
    COUNTRY_CODE = "Country Code"
    NAME_OF_ENTITY = "Name of entity"
    ADDRESS_OF_ENTITY = "Address of entity"
    ZIP_CODE = "zip code"
    NATURE_OF_ENTITY = "Nature of entity"
    DATE = "Date of acquiring interest/ stake"
    INITIAL_VALUE = "Initial value of investment"
    PEAK_VALUE = "Peak value of investment"
    CLOSING_BALANCE = "Closing Balance"
    GROSS_AMOUNT = "Total gross amount paid/credited with respect to the holding during the period"
    GROSS_PROCEEDS = "Total gross proceeds from sale or redemption of investment during the period"


MONTH_NUM_ABBR_LOOKUP = {
    "01": "Jan",
    "02": "Feb",
    "03": "Mar",
    "04": "Apr",
    "05": "May",
    "06": "Jun",
    "07": "Jul",
    "08": "Aug",
    "09": "Sep",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec",
}

def lookup_month_abbr(month: str) -> str:
    """Lookup month abbreviation from month number.
    month: str: month number in string format (e.g. "01")
    """
    return MONTH_NUM_ABBR_LOOKUP[month]

In [ ]:
import pandas as pd

df = pd.read_csv(FILE_PATH)
df.head()

## Preprocessing Data

Ensure the CSV file contains non-null values. If any are present, replace them with appropriate values. Additionally, review the data thoroughly and perform sanity checks to verify its accuracy and quality.

In [ ]:
print(f"report contains {len(df)} rows")

In [ ]:
# check null values in df
df.isnull().sum()

In [ ]:
# remove rows where "Country Name" is null
df = df.dropna(subset=[Fields.COUNTRY_NAME])

# remove rows where "Country Code" is null
df = df.dropna(subset=[Fields.COUNTRY_CODE])

print(f"# records after dropping null values: {len(df)}")

In [ ]:
# convert "Date of acquiring interest/ stake" column to date
df[Fields.DATE] = pd.to_datetime(df[Fields.DATE], errors="coerce")

# replace value "-" by 0 in columns
# "Total gross amount paid/credited with respect to the holding during the period"
# and "Total gross proceeds from sale or redemption of investment during the period"
df.replace({Fields.GROSS_PROCEEDS: "-"}, 0, inplace=True)
df.replace({Fields.GROSS_AMOUNT: "-"}, 0, inplace=True)

# convert columns to int
# * "Initial value of investment"
# * "Peak value of investment"
# * "Closing Balance"
# * "Total gross amount paid/credited with respect to the holding during the period"
# * and "Total gross proceeds from sale or redemption of investment during the period"

df[Fields.INITIAL_VALUE] = df[Fields.INITIAL_VALUE].astype(int)
df[Fields.PEAK_VALUE] = df[Fields.PEAK_VALUE].astype(int)
df[Fields.CLOSING_BALANCE] = df[Fields.CLOSING_BALANCE].astype(int)
df[Fields.GROSS_AMOUNT] = df[Fields.GROSS_AMOUNT].astype(int)
df[Fields.GROSS_PROCEEDS] = df[Fields.GROSS_PROCEEDS].astype(int)

In [ ]:
df

## Connect selenium with existing browser session

**1. Open chrome browser in debugging mode:**

**On Windows:**
   - Open terminal and navigate to the directory where you have the Chrome browser file (chrome.exe in case of windows).
   - Execute the following command: `chrome --remote-debugging-port=9222`. This will open the Chrome browser in debugging mode.

**On macOS:**

To open Chrome in debugging mode, you can use the following command:

```bash
/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222 --user-data-dir=/tmp/chrome_dev_test
```
- **Note**: You can use any port number other than 9222 if it's already in use.

---

**2. Login to the ITR portal in the opened window:**
   - Login to the ITR portal. (https://www.incometax.gov.in)
   - Complete the steps and open the **Schedule FA** page in the ITR-2 form.
   - Please delete if there is any existing entry if required.
   - Make sure to do the first entry manually (add a dummy entry with values null or 0, and once the data processing is completed, delete the dummy entry).
   - Your page should look `A3` section collapsed and ready to add a new entry.

---

**Note:**
- Notice that the **A3** section is at collapsed state in your web browser window. This is important because the section gets auto collpased when you succesfuly add a row to this section.

- Delete the dummy entry you created once the data processing is completed i.e., `Publish Data to ITR Schedule FA :: Table A3`.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep

In [ ]:
# creating webdriver object
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
chrome_options.binary_location = BROWSER_PATH
chrome_options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration

service = Service(executable_path=CHROME_DRIVER_PATH)

print("Connecting to Chrome...")
driver = webdriver.Chrome(service=service, options=chrome_options)
print("Connected...")

try:
    print(driver.title)
except Exception as e:
    print("Error:", e)

## Publish Data to ITR Schedule FA :: Table A3

The following cell iterates through each row in the dataframe and performs the following steps:

- Reads values from the columns _(ensure you modify the column names to match those in your CSV file)_.
- Retrieves the corresponding webpage elements for each column.
- Populates the UI fields with the values from the row.
- Outputs the row number once the row is successfully processed.


**Note:**
- The time required for this process depends on the number of rows in your file.
- The variable `idx_start` specifies the starting row number for data entry. This is useful if you need to `rerun` the script/cell after an error. Set `idx_start` to the row number where you want to resume data entry.

In [ ]:
idx_start = 0

class Record:
    def __init__(self, row):
        self.country = row[Fields.COUNTRY_NAME]
        self.name_of_entity = row[Fields.NAME_OF_ENTITY]
        self.address_of_entity = row[Fields.ADDRESS_OF_ENTITY]
        self.zip_code = str(row[Fields.ZIP_CODE])
        self.nature_of_entity = row[Fields.NATURE_OF_ENTITY]
        self.date = str(row[Fields.DATE])
        self.init_value = str(row[Fields.INITIAL_VALUE])
        self.peak_value = str(row[Fields.PEAK_VALUE])
        self.closing_value = str(row[Fields.CLOSING_BALANCE])
        self.gross_amount = str(row[Fields.GROSS_AMOUNT])
        self.gross_proceeds = str(row[Fields.GROSS_PROCEEDS])


for idx in range(idx_start, len(df)):
    record = Record(df.iloc[idx])
    # click on 'A3' table
    a3_table = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_hyper_490"]/div/div[1]/div/div/span')
    a3_table.click()
    sleep(2)

    # click on 'Add Another' button
    add_another_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_button_330"]')
    sleep(1)
    add_another_element.click()

    # enabling sleep to load the UI elements properly
    sleep(5)
    # get all elements from UI
    name_of_entity_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_203"]')
    address_of_entity_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_204"]')
    zip_code_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_205"]')
    nature_of_entity_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_206"]')
    init_value_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_207"]')
    peak_value_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_208"]')
    closing_value_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_209"]')
    gross_amount_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_210"]')
    gross_proceeds_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_211"]')
    date_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_471"]')
    country_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_dropdown_164"]')

    # Add button element
    add_element = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_button_279"]')

    sleep(5)

    country_select = Select(country_element)
    country_select.select_by_visible_text(record.country)

    # Put value into all other elements at UI
    name_of_entity_element.send_keys(record.name_of_entity)
    address_of_entity_element.send_keys(record.address_of_entity)
    zip_code_element.send_keys(record.zip_code)
    nature_of_entity_element.send_keys(record.nature_of_entity)
    init_value_element.send_keys(record.init_value)
    peak_value_element.send_keys(record.peak_value)
    closing_value_element.send_keys(record.closing_value)
    gross_amount_element.send_keys(record.gross_amount)
    gross_proceeds_element.send_keys(record.gross_proceeds)

    # set date value into date element in UI
    date_arr = record.date.split("-")
    year, month, day = date_arr[0], lookup_month_abbr(date_arr[1]), date_arr[2]
    date_element.click()
    # select year, month and day from the date picker
    month_year_element = driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/table/thead/tr[2]/th[1]")
    month_year_element.click()
    year_element = driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/table/thead/tr[2]/th[1]")
    year_element.click()
    year_box = driver.find_element(By.XPATH, f"//*[text()='{year}']")
    year_box.click()
    month_box = driver.find_element(By.XPATH, f"//*[text()='{month}']")
    month_box.click()
    day_box = driver.find_element(By.XPATH, f"//td[@class='day' and text()='{int(day.split(' ')[0])}']")
    day_box.click()

    # click on 'Add' button to add all the filled details
    add_element.click()

    print(f"idx: {idx} copied successfully.")
    sleep(2)

print("All the records are successfully added to the ITR Schedule FA. Please cross verify once before proceeding further!")

## Troubleshooting

If the above cell fails, follow these steps to ensure the page is returned to its previous state (as described earlier, with section A3 collapsed) and proceed as outlined below until the issue is resolved:

1. The cell will output the last successfully processed row number. Set the idx_start value to this number plus 1, and run the cell again. For example, if the last successful row was 10, set `idx_start = 11` and rerun the cell to resume data entry from the point of failure.
2. If the error recurs, identify the failing line and add `sleep()` before it, or increase the `sleep()` duration before rerunning the cell.